<p style="font-size:200%; font-weight:bold"> Whale New </p>

- Author: Yichen Luo
- Date: 20220930

In [22]:
import pandas as pd
import threading
import requests
import time
import os
import datetime
import re
import string
import random
import numpy as np
from web3 import Web3
from hexbytes import HexBytes
import seaborn as sns ; sns.set_theme(color_codes=True)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from IPython.core.interactiveshell import InteractiveShell
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)

RD = '/home/team_defi/02_rdata/'
WD = '/home/team_defi/03_wdata/'
RES = '/home/team_defi/04_result/'
InteractiveShell.ast_node_interactivity = 'all'

# Function

In [2]:
def contract(address, chian):
    
    if chain == 'ethereum': rpc = 'https://rpc.ankr.com/eth'
    if chain == 'binance': rpc = 'https://bscrpc.com'
    if chain == 'avalanche': rpc = 'https://api.avax.network/ext/bc/C/rpc'
    if chain == 'fantom': rpc = 'https://rpc.ftm.tools'
    if chain == 'polygon': rpc = 'https://polygon-rpc.com'

    w3 = Web3(Web3.HTTPProvider(rpc))
    contract = w3.eth.getCode(w3.toChecksumAddress(address)).hex() != '0x'

    return contract

In [3]:
def chain_info(chain):
    dexes = {'ethereum':['uniswapv3_ethereum', 'uniswapv2_ethereum', 'sushiswap_ethereum'],
             'binance':['pancake_binance', 'biswap_binance', 'apeswap_binance', 'babyswap_binance', 'jetswap_binance'],
             'avalanche':['traderjoe_avalanche', 'pangolin_avalanche'],
             'fantom':['spookyswap_fantom', 'spiritswap_fantom'],
             'polygon':['sushiswap_polygon', 'apeswap_polygon', 'jetswap_polygon']
            }
    
    dex = dexes[chain]
    return dex

In [4]:
def insider_transaction(address, dex):
    

    dfs = []
    for role in dex_info(dex)['role']:

        post = {"query":dex_info(dex)['post'].substitute({'role': role, 'address':address})}
        url = dex_info(dex)['url']

        r = requests.post(url, json=post)
        data = r.json()
        df = pd.DataFrame.from_dict(data['data']['swaps'])

        if df.shape[0] == 0:
            continue

        if dex_info(dex)['style'] == 0:
            df['token0_name'] = df['token0'].apply(lambda x: x['name'])
            df['token1_name'] = df['token1'].apply(lambda x: x['name'])
            df['token0_id'] = df['token0'].apply(lambda x: x['id'])
            df['token1_id'] = df['token1'].apply(lambda x: x['id'])
            df['transaction_id'] = df['transaction'].apply(lambda x: x['id'])
            df.drop(columns=['token0', 'token1', 'transaction'], inplace=True)
        else:
            if dex_info(dex)['style'] == 1:
                df['token0_name'] = df['pair'].apply(lambda x: x['token0']['name'])
                df['token1_name'] = df['pair'].apply(lambda x: x['token1']['name'])
                df['token0_id'] = df['pair'].apply(lambda x: x['token0']['id'])
                df['token1_id'] = df['pair'].apply(lambda x: x['token1']['id'])
                df['transaction_id'] = df['transaction'].apply(lambda x: x['id'])
                df.drop(columns=['pair', 'transaction'], inplace=True)
        
        df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s', origin='unix')
        
        if dex == 'uniswapv3_ethereum': df = uniswapv3_correction(df)
        
        df = df.loc[df[role] == address]
        if df.shape[0] == 0:
            continue

        dfs.append(df)

    if len(dfs) != 0: 
        dfs = pd.concat(dfs)
        dfs.drop_duplicates('id', inplace=True)

    return dfs

In [5]:
def uniswapv3_correction(df):
    
    df['amount0'] = df['amount0'].astype('float')
    df['amount1'] = df['amount1'].astype('float')
    for idx, row in df.iterrows():
        if row['amount0'] > 0:
            df.loc[df['id'] == row['id'], 'amount0Out'] = 0
            df.loc[df['id'] == row['id'], 'amount0In'] = abs(row['amount0'])
            df.loc[df['id'] == row['id'], 'amount1Out'] = abs(row['amount1'])
            df.loc[df['id'] == row['id'], 'amount1In'] = 0

        else:
            df.loc[df['id'] == row['id'], 'amount0Out'] = abs(row['amount0'])
            df.loc[df['id'] == row['id'], 'amount0In'] = 0
            df.loc[df['id'] == row['id'], 'amount1Out'] = 0
            df.loc[df['id'] == row['id'], 'amount1In'] = abs(row['amount1'])
    df.drop(columns=['amount1', 'amount0'], inplace=True)
    return df

In [6]:
def dex_info(dex):
    dex_infos = {
                'uniswapv3_ethereum':{'url': 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
                                       'role': 1,
                                       'style': 0,
                                       'post': 1
                                      },
                
                'uniswapv2_ethereum':{'url': 'https://api.thegraph.com/subgraphs/name/ianlapham/uniswapv2',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'sushiswap_ethereum':{'url': 'https://api.thegraph.com/subgraphs/name/sushiswap/exchange',
                                       'role': 2,
                                       'style': 1,
                                       'post': 2
                                      },
        
                'pancake_binance':{'url': 'https://bsc.streamingfast.io/subgraphs/name/pancakeswap/exchange-v2',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'biswap_binance':{'url': 'https://api.thegraph.com/subgraphs/name/biswapcom/exchange5',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'apeswap_binance':{'url': 'https://graph2.apeswap.finance/subgraphs/name/ape-swap/apeswap-subgraph',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'babyswap_binance':{'url': 'https://api.thegraph.com/subgraphs/name/loremboard/bsc-babyswap-subgraph',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'jetswap_binance':{'url': 'https://api.thegraph.com/subgraphs/name/smartcookie0501/jetswap-subgraph-v2',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'traderjoe_avalanche':{'url': 'https://api.thegraph.com/subgraphs/name/traderjoe-xyz/exchange',
                                       'role': 2,
                                       'style': 1,
                                       'post': 2
                                      }, 
        
                'pangolin_avalanche':{'url': 'https://api.thegraph.com/subgraphs/name/dasconnor/pangolin-dex',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },  
        
                'spookyswap_fantom':{'url': 'https://api.thegraph.com/subgraphs/name/int3grtor/spooky-exchange',
                                       'role': 2,
                                       'style': 1,
                                       'post': 2
                                      },  
        
                'spiritswap_fantom':{'url': 'https://api.thegraph.com/subgraphs/name/evele/spirit-swap-subgraph-dev-v2',
                                       'role': 2,
                                       'style': 1,
                                       'post': 2
                                      },  
        
                'sushiswap_polygon':{'url': 'https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange',
                                       'role': 2,
                                       'style': 1,
                                       'post': 2
                                      },
        
                'apeswap_polygon':{'url': 'https://api.thegraph.com/subgraphs/name/apeswapfinance/dex-polygon',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
        
                'jetswap_polygon':{'url': 'https://api.thegraph.com/subgraphs/name/smartcookie0501/jetswap-subgraph-polygon-v2',
                                       'role': 0,
                                       'style': 1,
                                       'post': 0
                                      },
                }
    
    roles = [['from', 'to', 'sender'],
             ['origin', 'recipient', 'sender'],
             ['sender', 'to']]
    
    url = dex_infos[dex]['url']
    role = roles[dex_infos[dex]['role']]
    style = dex_infos[dex]['style']
    post = dex_infos[dex]['post']
    
    info = {'url':url,
            'post':post_string(post),
            'role':role,
            'style':style
           }

    return info

In [7]:
def post_string(post):
    if post == 0:
        query = string.Template("""query MyQuery {
  swaps(first: 1000, where: {$role: "$address"}) {
    amount0In
    amount0Out
    amount1In
    amount1Out
    amountUSD
    from
    id
    logIndex
    to
    timestamp
    sender
    pair {
      token0 {
        id
        name
      }
      token1 {
        id
        name
      }
    }
    transaction {
      id
    }
  }
}""")
        
    if post == 1:
        query = string.Template("""query MyQuery {
  swaps(first: 1000, where: {$role: "$address"}) {
    amount0
    amount1
    amountUSD
    id
    logIndex
    origin
    recipient
    sender
    sqrtPriceX96
    tick
    timestamp
    token0 {
      id
      name
    }
    token1 {
      id
      name
    }
    transaction {
      id
    }
  }
}""")
        
    if post == 2:
        query = string.Template("""query MyQuery {
  swaps(first: 1000, where: {$role: "$address"}) {
    amount0In
    amount0Out
    amount1In
    amount1Out
    amountUSD
    id
    logIndex
    sender
    timestamp
    to
    pair {
      token0 {
        id
        name
      }
      token1 {
        id
        name
      }
    }
    transaction {
      id
    }
  }
}""")
    return query

In [12]:
def home_token_filter(df, insider_contractAddress):
    df['home_token'] = False
    df['token0_id'] = df['token0_id'].apply(lambda x: x.lower())
    df['token1_id'] = df['token1_id'].apply(lambda x: x.lower())
    df.loc[(df['token0_id'] == insider_contractAddress.lower())|(df['token1_id'] == insider_contractAddress.lower()), 'home_token'] = True
    return df

In [19]:
def liquidity_provision(transaction_id, chain):
    
    time.sleep(random.random() * 2)
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}

    if chain == 'ethereum': url = f'https://etherscan.io/tx/{transaction_id}'
    if chain == 'binance' : url = f'https://bscscan.com/tx/{transaction_id}'
    if chain == 'avalanche': url = f'https://snowtrace.io/tx/{transaction_id}'
    if chain == 'polygon' : url = f'https://polygonscan.com/tx/{transaction_id}'
    if chain == 'fantom' : url = f'https://ftmscan.com/tx/{transaction_id}'

    r = requests.get(url, headers=headers)
    if re.search('liquidity to', r.text.lower()) != None:
        return True
    else:
        return False

In [25]:
def set_buy(df):

    
    for idx, row in df.iterrows():
        index = row['id']
        insider_contractAddress = row['insider_contractAddress'].lower()
        token0_id = row['token0_id'].lower()
        token1_id = row['token1_id'].lower()

        amount0Out = row['amount0Out']
        amount0In = row['amount0In']
        amount1Out = row['amount1Out']
        amount1In = row['amount1In']
        amountUSD = row['amountUSD']

        if insider_contractAddress == token0_id:
            if (amount0In == 0)|(amount1Out == 0):
                df.loc[df['id'] == index, 'buy'] = amountUSD
            else:
                df.loc[df['id'] == index, 'buy'] = - amountUSD

        else:
            if (amount0In == 0)|(amount1Out == 0):
                df.loc[df['id'] == index, 'buy'] = - amountUSD
            else:
                df.loc[df['id'] == index, 'buy'] = amountUSD
            
    return df

In [28]:
def token_return(after, day, contract, chain, timestamp):
    if after == True:
        start_date = pd.to_datetime(timestamp)
        end_date = pd.to_datetime(timestamp) + datetime.timedelta(days=day)
        
        start_price = token_dollar_price(contract=contract, chain=chain, timestamp=start_date)
        end_price = token_dollar_price(contract=contract, chain=chain, timestamp=end_date)
        
    else:
        start_date = pd.to_datetime(timestamp) - datetime.timedelta(days=day+1)
        end_date = pd.to_datetime(timestamp) - datetime.timedelta(days=1)
        
        start_price = token_dollar_price(contract=contract, chain=chain, timestamp=start_date)
        end_price = token_dollar_price(contract=contract, chain=chain, timestamp=end_date)
    
    if (end_price == 0)|(start_price==0):
        return np.nan
    else:
        return np.log(end_price/start_price)
    
def token_return_date(after, day, contract, chain, date):
    if after == True:
        start_date = pd.to_datetime(date)
        end_date = pd.to_datetime(date) + datetime.timedelta(days=day)
        
        start_price = token_dollar_price_date(contract=contract, chain=chain, date=start_date)
        end_price = token_dollar_price_date(contract=contract, chain=chain, date=end_date)
        
    else:
        start_date = pd.to_datetime(date) - datetime.timedelta(days=day+1)
        end_date = pd.to_datetime(date) - datetime.timedelta(days=1)
        
        start_price = token_dollar_price_date(contract=contract, chain=chain, date=start_date)
        end_price = token_dollar_price_date(contract=contract, chain=chain, date=end_date)
    
    if (end_price == 0)|(start_price==0):
        return np.nan
    else:
        return np.log(end_price/start_price)

In [29]:
def token_dollar_price(contract, chain, timestamp):
    
        contractAddress_lower = contract.lower()
        contractAddress_upper = Web3.toChecksumAddress(contractAddress_lower)

        date = timestamp.strftime('%Y-%m-%d')
        
        priceUSD = 0
        dex_num = 0
        avg_priceUSD = 0

        for dex in chain_info(chain):
            lst = pd.read_csv(f'{WD}/dex_price/list_all/{dex}.csv')
            csv_info = lst.loc[((lst['contractAddress'] == contractAddress_lower)|
                                (lst['contractAddress'] == contractAddress_upper))]
            
            if csv_info['contractAddress'].values[0] == contractAddress_lower:
                contractAddress = contractAddress_lower
            else:
                contractAddress = contractAddress_upper

            status = csv_info['status'].values[0]
            dexn = csv_info['dex'].values[0]

            if status == 0: 
                continue
            elif dexn == 0:
                continue
            else:
                csv = pd.read_csv(f'{WD}dex_price/csv_all/{contractAddress}_{dex}.csv') 
                price_info = csv.loc[(pd.to_datetime(csv['date']) > pd.to_datetime(timestamp) - datetime.timedelta(days=1))&
                                  (pd.to_datetime(csv['date']) < pd.to_datetime(timestamp))]
                
                if price_info.shape[0] != 0:
                    price = price_info['priceUSD'].values[0]
                    if price != 0:  
                        priceUSD += price
                        dex_num += 1

        if dex_num != 0: 
            return priceUSD/dex_num
        if dex_num == 0:
            return 0
        
def token_dollar_price_date(contract, chain, date):
    
        contractAddress_lower = contract.lower()
        contractAddress_upper = Web3.toChecksumAddress(contractAddress_lower)
        
        priceUSD = 0
        dex_num = 0
        avg_priceUSD = 0

        for dex in chain_info(chain):
            lst = pd.read_csv(f'{WD}/dex_price/list_all/{dex}.csv')
            csv_info = lst.loc[((lst['contractAddress'] == contractAddress_lower)|
                                (lst['contractAddress'] == contractAddress_upper))]
            
            if csv_info['contractAddress'].values[0] == contractAddress_lower:
                contractAddress = contractAddress_lower
            else:
                contractAddress = contractAddress_upper

            status = csv_info['status'].values[0]
            dexn = csv_info['dex'].values[0]

            if status == 0: 
                continue
            elif dexn == 0:
                continue
            else:
                csv = pd.read_csv(f'{WD}dex_price/csv_all/{contractAddress}_{dex}.csv') 
                price_info = csv.loc[(pd.to_datetime(csv['date']) == pd.to_datetime(date))]
                
                if price_info.shape[0] != 0:
                    price = price_info['priceUSD'].values[0]
                    if price != 0:  
                        priceUSD += price
                        dex_num += 1

        if dex_num != 0: 
            return priceUSD/dex_num
        if dex_num == 0:
            return 0

# New Definition

- More than 2%
- Non-root
- Non-layer-one
- Non-contract

## List

In [29]:
lst = pd.read_csv(f'{WD}whale/list/panel.csv')
lst.shape[0]
lst.loc[lst['status'] == -1].shape[0]

13509

301

## Example

### Dup Check

In [30]:
contractAddress = '0x05732b6D78591E75B31022d71E55Aa810498A5a4'
chain = 'binance'

df = pd.read_csv(f'{WD}whale/csv/{contractAddress}_{chain}.csv')
df['Percentage'] = df['Percentage'].apply(lambda x: float(x.strip('%'))/100)
df.loc[df['Percentage'] > 0.02]
df

df = df.loc[[True if (re.match('0x', i) != None) else False for i in df['Address']]]
df

,Rank,Address,Quantity,Percentage
0,1,Null Address: 0x000...dEaD,"37,140,779,846,902,900.079626967",0.371408
1,2,0x8d90ce5dfa793418b55e650b3939f7a6c718d749,"13,097,654,138,813,600.821411543",0.130977
2,3,PancakeSwap V2: SHiBGOTCHi 6,"9,159,703,720,662,800.506592372",0.091597
3,4,0xac463a08dfc47958e1c385a8becc3a6f0537b50d,"4,092,793,901,543,380.267444078",0.040928
4,5,HODL: LBank LP,"3,887,104,627,157,750.439164142",0.038871
5,6,0x23e6863c9cf9aec0eae45652ff82510ef7394740,"2,332,784,674,279,820.828056586",0.023328
6,7,0x490027e4876a4a61f415fec8a95265486d8c47bf,"2,238,148,368,254,320.304345007",0.022381


,Rank,Address,Quantity,Percentage
0,1,Null Address: 0x000...dEaD,"37,140,779,846,902,900.079626967",0.371408
1,2,0x8d90ce5dfa793418b55e650b3939f7a6c718d749,"13,097,654,138,813,600.821411543",0.130977
2,3,PancakeSwap V2: SHiBGOTCHi 6,"9,159,703,720,662,800.506592372",0.091597
3,4,0xac463a08dfc47958e1c385a8becc3a6f0537b50d,"4,092,793,901,543,380.267444078",0.040928
4,5,HODL: LBank LP,"3,887,104,627,157,750.439164142",0.038871
...,...,...,...,...
95,96,0x8909cc1b7d8f96aca41955923458b91f7d06c224,"22,746,851,254,572.20816185",0.000227
96,97,0x992ee9317b7932572fbe61a38de63d12f913b572,"21,881,838,945,779.298125137",0.000219
97,98,0xa8bebae05029bb487171f4640b278f58fda2a2ec,"21,474,740,276,749.994353874",0.000215
98,99,0xfbba7bb04f226a0cc6323316fc19d8c3b0a2af58,"20,833,377,154,882.679666753",0.000208


,Rank,Address,Quantity,Percentage
1,2,0x8d90ce5dfa793418b55e650b3939f7a6c718d749,"13,097,654,138,813,600.821411543",0.130977
3,4,0xac463a08dfc47958e1c385a8becc3a6f0537b50d,"4,092,793,901,543,380.267444078",0.040928
5,6,0x23e6863c9cf9aec0eae45652ff82510ef7394740,"2,332,784,674,279,820.828056586",0.023328
6,7,0x490027e4876a4a61f415fec8a95265486d8c47bf,"2,238,148,368,254,320.304345007",0.022381
7,8,0x48f862c160a408e8bd960e860343eb80e3a2c3ea,"1,954,907,934,032,570.751834197",0.019549
...,...,...,...,...
95,96,0x8909cc1b7d8f96aca41955923458b91f7d06c224,"22,746,851,254,572.20816185",0.000227
96,97,0x992ee9317b7932572fbe61a38de63d12f913b572,"21,881,838,945,779.298125137",0.000219
97,98,0xa8bebae05029bb487171f4640b278f58fda2a2ec,"21,474,740,276,749.994353874",0.000215
98,99,0xfbba7bb04f226a0cc6323316fc19d8c3b0a2af58,"20,833,377,154,882.679666753",0.000208


## More than 2%

In [31]:
lst = pd.read_csv(f'{WD}whale/list/panel.csv')
dfs = []

for idx, row in lst.loc[lst['status'] >0].iterrows():
    contractAddress = row['contractAddress']
    chain = row['chain']
    tokenName = row['tokenName']
    tokenSymbol = row['tokenSymbol']

    df = pd.read_csv(f'{WD}whale/csv/{contractAddress}_{chain}.csv')
    df = df.loc[[True if (re.match('0x', i) != None) else False for i in df['Address']]]
    
    df['percentage'] = df['Percentage'].apply(lambda x: float(x.strip('%').replace(',', ''))/100)
    df['tokenName'] = tokenName
    df['tokenSymbol'] = tokenSymbol
    df['contractAddress'] = contractAddress
    df['chain'] = chain
    df = df.loc[df['percentage'] > 0.02]
    dfs.append(df)

    
dfs = pd.concat(dfs)
dfs

,Rank,Address,Quantity,Percentage,percentage,tokenName,tokenSymbol,contractAddress,chain
0,1,0x3b5bc2643640c87235ff00160c15d31539f02a94,"960,409,000,000,000",96.0409%,0.960409,Callisto,C3000,0x2a15de7c821baa1a727039b76d540b36b23db257,ethereum
0,1,0x990d3a9832928ebc7c6cbe7261dc626000c23775,"960,409,000,000,000",96.0409%,0.960409,Callisto,C3000,0xa0fd12272cc4da86aebf9d592cdad253cbbc25fb,ethereum
0,1,0x755f60275578d747763f098f22722fc6b66c6730,"24,428,251.447237437586850749",15.7005%,0.157005,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum
2,3,0x5ed7d8e2089b2b0e15439735b937cec5f0ae811b,"9,028,385.664053631184250482",5.8027%,0.058027,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum
3,4,0x7f87aaccbafd3c1ac66a84aba719de3dee7230b7,"7,909,417.30403295928887199",5.0835%,0.050835,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum
...,...,...,...,...,...,...,...,...,...
7,8,0x3cb4e9fafe18bec85c6aad0dcc0b8b5f8d6095ed,"289,246.541555648",2.8925%,0.028925,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon
8,9,0x89c62f31bec91a6a7c21d09f9963cad7781d29b2,"238,115.249059482",2.3812%,0.023812,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon
9,10,0x75ecf51c22e94a20164075fc5d7faff956ca6640,"223,212.385068043",2.2321%,0.022321,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon
0,1,0xadc42c844b8392ca353e079030609b6303d7f51d,"597,712,066.605938534",59.7712%,0.597712,Poge,Poge,0x6EA856c7A96d7366De2bbB600B956617d344DDa4,polygon


In [32]:
dfs.to_csv(f'{WD}whale_new/list_raw/list.csv', index=False)

## Non-Root

In [33]:
lst = pd.read_csv(f'{WD}whale_new/list_raw/list.csv')
lst['root'] = False
lst['address'] = lst['Address'].apply(lambda x: x.lower())
lst

,Rank,Address,Quantity,Percentage,percentage,tokenName,tokenSymbol,contractAddress,chain,root,address
0,1,0x3b5bc2643640c87235ff00160c15d31539f02a94,"960,409,000,000,000",96.0409%,0.960409,Callisto,C3000,0x2a15de7c821baa1a727039b76d540b36b23db257,ethereum,False,0x3b5bc2643640c87235ff00160c15d31539f02a94
1,1,0x990d3a9832928ebc7c6cbe7261dc626000c23775,"960,409,000,000,000",96.0409%,0.960409,Callisto,C3000,0xa0fd12272cc4da86aebf9d592cdad253cbbc25fb,ethereum,False,0x990d3a9832928ebc7c6cbe7261dc626000c23775
2,1,0x755f60275578d747763f098f22722fc6b66c6730,"24,428,251.447237437586850749",15.7005%,0.157005,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x755f60275578d747763f098f22722fc6b66c6730
3,3,0x5ed7d8e2089b2b0e15439735b937cec5f0ae811b,"9,028,385.664053631184250482",5.8027%,0.058027,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x5ed7d8e2089b2b0e15439735b937cec5f0ae811b
4,4,0x7f87aaccbafd3c1ac66a84aba719de3dee7230b7,"7,909,417.30403295928887199",5.0835%,0.050835,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x7f87aaccbafd3c1ac66a84aba719de3dee7230b7
...,...,...,...,...,...,...,...,...,...,...,...
36981,8,0x3cb4e9fafe18bec85c6aad0dcc0b8b5f8d6095ed,"289,246.541555648",2.8925%,0.028925,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x3cb4e9fafe18bec85c6aad0dcc0b8b5f8d6095ed
36982,9,0x89c62f31bec91a6a7c21d09f9963cad7781d29b2,"238,115.249059482",2.3812%,0.023812,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x89c62f31bec91a6a7c21d09f9963cad7781d29b2
36983,10,0x75ecf51c22e94a20164075fc5d7faff956ca6640,"223,212.385068043",2.2321%,0.022321,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x75ecf51c22e94a20164075fc5d7faff956ca6640
36984,1,0xadc42c844b8392ca353e079030609b6303d7f51d,"597,712,066.605938534",59.7712%,0.597712,Poge,Poge,0x6EA856c7A96d7366De2bbB600B956617d344DDa4,polygon,False,0xadc42c844b8392ca353e079030609b6303d7f51d


In [34]:
chains = ['ethereum', 'binance', 'avalanche', 'fantom', 'polygon']
dfs = []
for chain in chains:
    df = pd.read_csv(f'/home/yichen/team_defi/04_result/insider_trading/root_list/root_list_final/root_{chain}.csv')
    for idx, row in df.iterrows():
        root = row['root'].lower()
        contractAddress = row['contractAddress']
        
        lst.loc[(lst['contractAddress'] == contractAddress)&(lst['chain'] == chain)&(lst['address'] == root), 'root'] = True

In [35]:
lst['root'].value_counts()
lst.loc[lst['root'] == True].sample(1)

False    28166
True      8820
Name: root, dtype: int64

,Rank,Address,Quantity,Percentage,percentage,tokenName,tokenSymbol,contractAddress,chain,root,address
634,1,0x8b3e86fc960936b5530b8d36ec022dc7fd198554,"551,026,421.411651845",55.1026%,0.551026,Chinu Coin,CHINU,0x39243cc473147469d3149fa267de26cbf64a05f0,ethereum,True,0x8b3e86fc960936b5530b8d36ec022dc7fd198554


In [36]:
df = pd.read_csv(f'/home/yichen/team_defi/04_result/insider_trading/root_list/root_list_final/root_ethereum.csv')
df.loc[df['root'] == '0x6c5265f6f52623c558dac043301514ea9fbcdbd0']

,root,tokenName,tokenSymbol,contractAddress
2987,0x6c5265f6f52623c558dac043301514ea9fbcdbd0,Oasis,OASIS,0xfb04c6435d5f346c3A7C8eCf9EF637b3Ce626AF0


In [41]:
lst = lst.loc[lst['root'] == False]
lst
lst.to_csv(f'{WD}whale_new/list_non_root/list.csv', index=False)

,Rank,Address,Quantity,Percentage,percentage,tokenName,tokenSymbol,contractAddress,chain,root,address
0,1,0x3b5bc2643640c87235ff00160c15d31539f02a94,"960,409,000,000,000",96.0409%,0.960409,Callisto,C3000,0x2a15de7c821baa1a727039b76d540b36b23db257,ethereum,False,0x3b5bc2643640c87235ff00160c15d31539f02a94
1,1,0x990d3a9832928ebc7c6cbe7261dc626000c23775,"960,409,000,000,000",96.0409%,0.960409,Callisto,C3000,0xa0fd12272cc4da86aebf9d592cdad253cbbc25fb,ethereum,False,0x990d3a9832928ebc7c6cbe7261dc626000c23775
2,1,0x755f60275578d747763f098f22722fc6b66c6730,"24,428,251.447237437586850749",15.7005%,0.157005,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x755f60275578d747763f098f22722fc6b66c6730
3,3,0x5ed7d8e2089b2b0e15439735b937cec5f0ae811b,"9,028,385.664053631184250482",5.8027%,0.058027,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x5ed7d8e2089b2b0e15439735b937cec5f0ae811b
4,4,0x7f87aaccbafd3c1ac66a84aba719de3dee7230b7,"7,909,417.30403295928887199",5.0835%,0.050835,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x7f87aaccbafd3c1ac66a84aba719de3dee7230b7
...,...,...,...,...,...,...,...,...,...,...,...
36980,7,0x6fa7ac8b3870e78ac694af711d795f72dffd17d1,"291,347.723592884",2.9135%,0.029135,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x6fa7ac8b3870e78ac694af711d795f72dffd17d1
36981,8,0x3cb4e9fafe18bec85c6aad0dcc0b8b5f8d6095ed,"289,246.541555648",2.8925%,0.028925,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x3cb4e9fafe18bec85c6aad0dcc0b8b5f8d6095ed
36982,9,0x89c62f31bec91a6a7c21d09f9963cad7781d29b2,"238,115.249059482",2.3812%,0.023812,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x89c62f31bec91a6a7c21d09f9963cad7781d29b2
36983,10,0x75ecf51c22e94a20164075fc5d7faff956ca6640,"223,212.385068043",2.2321%,0.022321,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x75ecf51c22e94a20164075fc5d7faff956ca6640


## Non-Layer-One

In [3]:
lo = pd.read_csv(f'{WD}connected_wallet/panel/panel_complete.csv')
lo
lo.to_csv(f'{RES}insider_trading/layer1_list/layer_one_list.csv', index=False)

,layer_one,insider_contractAddress,chain,amountUSD
0,0x000159831a681a63b01911b9c162fbb8949976ba,0x6670d7d4091d9c2f4fb915291630da9146490681,binance,36469.024594
1,0x0007a6a1eae3f82897f92f2e9fc476c4be8d4243,0xaf1167b1f90e4f27d9f520a4cd3a1e452e011cea,binance,2404.686263
2,0x0009343103cbe1d0898f6f19f0f9442770d008e4,0xbc23833d965632cec84bf98181cb84c31bbdfa94,binance,1356.067500
3,0x0010af1fef93215f345b3531f8d9a39efec93ffb,0x643ecd70611e240259e5974468dad69560c5c892,binance,5986.534576
4,0x0017177f12a5b8e8c0f13431f12811f5c523fea1,0x5c28dbdf5351089ca413b4493dd6c6043e44af42,ethereum,1556.044180
...,...,...,...,...
25033,0xdf9eec260da99a01b2b893ac512d05c6fbed83a6,0x0ccd575bf9378c06f6dca82f8122f570769f00c2,binance,1684.401378
25034,0xe84638119acd8b7c4c1de08d03027c7a46bd4ae1,0x0ccd575bf9378c06f6dca82f8122f570769f00c2,binance,2561.882619
25035,0xf41399aac0d78cc955108e12916204d90faff875,0x0ccd575bf9378c06f6dca82f8122f570769f00c2,binance,1946.795993
25036,0xfb29ace7a0d650969ca940ecfb0d904d13ddc23f,0x0ccd575bf9378c06f6dca82f8122f570769f00c2,binance,1201.676021


In [43]:
lst = pd.read_csv(f'{WD}whale_new/list_non_root/list.csv')
lst['layer_one'] = False

for chain in ['ethereum', 'binance', 'avalanche', 'polygon']:
    for idx, row in lo.loc[lo['chain'] == chain].iterrows():
        layer_one = row['layer_one'].lower()
        insider_contractAddress = row['insider_contractAddress']
        
        lst.loc[(lst['contractAddress'] == insider_contractAddress)&(lst['chain'] == chain)&(lst['address'] == layer_one), 'layer_one'] = True

In [46]:
lst['layer_one'].value_counts()
lst.loc[lst['layer_one'] == True].sample(1)

False    27934
True       232
Name: layer_one, dtype: int64

,Rank,Address,Quantity,Percentage,percentage,tokenName,tokenSymbol,contractAddress,chain,root,address,layer_one
130,4,0x9bb547f62dd221286787320e13804316ddadb14d,"21,077,522,868,928.771354236259010443",3.5941%,0.035941,Women Empowerment Token,WEMP,0x79e52c8d2ca6ad34791899fcd19752a8bc51dea5,ethereum,False,0x9bb547f62dd221286787320e13804316ddadb14d,True


In [47]:
lo.loc[lo['layer_one'] == '0x9bb547f62dd221286787320e13804316ddadb14d']

,layer_one,insider_contractAddress,chain,amountUSD
6862,0x9bb547f62dd221286787320e13804316ddadb14d,0x79e52c8d2ca6ad34791899fcd19752a8bc51dea5,ethereum,486925.245377


In [48]:
lst = lst.loc[lst['layer_one'] == False]
lst
lst.to_csv(f'{WD}whale_new/list_non_layer_one/list.csv', index=False)

,Rank,Address,Quantity,Percentage,percentage,tokenName,tokenSymbol,contractAddress,chain,root,address,layer_one
0,1,0x3b5bc2643640c87235ff00160c15d31539f02a94,"960,409,000,000,000",96.0409%,0.960409,Callisto,C3000,0x2a15de7c821baa1a727039b76d540b36b23db257,ethereum,False,0x3b5bc2643640c87235ff00160c15d31539f02a94,False
1,1,0x990d3a9832928ebc7c6cbe7261dc626000c23775,"960,409,000,000,000",96.0409%,0.960409,Callisto,C3000,0xa0fd12272cc4da86aebf9d592cdad253cbbc25fb,ethereum,False,0x990d3a9832928ebc7c6cbe7261dc626000c23775,False
2,1,0x755f60275578d747763f098f22722fc6b66c6730,"24,428,251.447237437586850749",15.7005%,0.157005,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x755f60275578d747763f098f22722fc6b66c6730,False
3,3,0x5ed7d8e2089b2b0e15439735b937cec5f0ae811b,"9,028,385.664053631184250482",5.8027%,0.058027,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x5ed7d8e2089b2b0e15439735b937cec5f0ae811b,False
4,4,0x7f87aaccbafd3c1ac66a84aba719de3dee7230b7,"7,909,417.30403295928887199",5.0835%,0.050835,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x7f87aaccbafd3c1ac66a84aba719de3dee7230b7,False
...,...,...,...,...,...,...,...,...,...,...,...,...
28161,7,0x6fa7ac8b3870e78ac694af711d795f72dffd17d1,"291,347.723592884",2.9135%,0.029135,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x6fa7ac8b3870e78ac694af711d795f72dffd17d1,False
28162,8,0x3cb4e9fafe18bec85c6aad0dcc0b8b5f8d6095ed,"289,246.541555648",2.8925%,0.028925,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x3cb4e9fafe18bec85c6aad0dcc0b8b5f8d6095ed,False
28163,9,0x89c62f31bec91a6a7c21d09f9963cad7781d29b2,"238,115.249059482",2.3812%,0.023812,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x89c62f31bec91a6a7c21d09f9963cad7781d29b2,False
28164,10,0x75ecf51c22e94a20164075fc5d7faff956ca6640,"223,212.385068043",2.2321%,0.022321,POKI,POKI,0x3E41CE4f7ef7ca7C23640d2C8a906552a2A90092,polygon,False,0x75ecf51c22e94a20164075fc5d7faff956ca6640,False


## Non-Contract

In [63]:
# lst = pd.read_csv(f'{WD}whale_new/list_non_layer_one/list.csv')
# lst['status'] = -1
# lst.to_csv(f'{WD}whale_new/list_contract/list.csv', index=False)

In [72]:
# lst = pd.read_csv(f'{WD}whale_new/list_contract/list.csv')
total = lst.shape[0]
done = lst.loc[lst['status'] != -1].shape[0]

for idx, row in lst.loc[lst['status'] == -1].drop_duplicates(['Address', 'chain']).iterrows():
    done += 1
    Address = row['Address']
    chain = row['chain']
    try:
        contract_status = contract(address=Address,chian=chain)
        if contract_status:
            lst.loc[(lst['Address'] == Address)&(lst['chain'] == chain), 'status'] = 1
            lst.to_csv(f'{WD}whale_new/list_contract/list.csv', index=False)
            print(f'{Address}, {chain} {contract_status} {done}/{total}')
        else:
            lst.loc[(lst['Address'] == Address)&(lst['chain'] == chain), 'status'] = 0
            lst.to_csv(f'{WD}whale_new/list_contract/list.csv', index=False)
            print(f'{Address}, {chain} {contract_status} {done}/{total}')
        
    except Exception as e:
        print(f'{Address}, {chain} Error: {e} {done}/{total}')

0xB1AdceddB2941033a090dD166a462fe1c2029484, ethereum False 23200/23200


In [71]:
lst['status'].value_counts()
lst.loc[lst['status'] == -1, 'Address'] = '0xB1AdceddB2941033a090dD166a462fe1c2029484'
lst.loc[lst['status'] == -1, 'address'] = '0xB1AdceddB2941033a090dD166a462fe1c2029484'.lower()
lst.loc[lst['status'] == -1]

 0    19609
 1     3590
-1        1
Name: status, dtype: int64

,Rank,Address,Quantity,Percentage,percentage,tokenName,tokenSymbol,contractAddress,chain,root,address,layer_one,status,contract
130,3,0xB1AdceddB2941033a090dD166a462fe1c2029484,"24,210",2.4210%,0.02421,Bemil.io,BEM,0x3102931098e75111a6f57cdf9a3e68dd007e5dd1,ethereum,False,0xb1adceddb2941033a090dd166a462fe1c2029484,False,-1,NaN


In [5]:
lst = pd.read_csv(f'{WD}whale_new/list_contract/list.csv')
lst = lst.loc[lst['status'] == 0]
lst
lst.to_csv(f'{RES}insider_trading/whale_list/list.csv', index=False)

,Rank,Address,Quantity,Percentage,percentage,tokenName,tokenSymbol,contractAddress,chain,root,address,layer_one,status,contract
3,3,0x5ed7d8e2089b2b0e15439735b937cec5f0ae811b,"9,028,385.664053631184250482",5.8027%,0.058027,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x5ed7d8e2089b2b0e15439735b937cec5f0ae811b,False,0,False
4,4,0x7f87aaccbafd3c1ac66a84aba719de3dee7230b7,"7,909,417.30403295928887199",5.0835%,0.050835,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x7f87aaccbafd3c1ac66a84aba719de3dee7230b7,False,0,False
5,5,0x36c39434db7987cae67233d06de9427c4c67e787,"6,075,739.162088075229878178",3.9050%,0.039050,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x36c39434db7987cae67233d06de9427c4c67e787,False,0,False
6,6,0x1e6da5ed1586271ef004cefac694eedfe77bc7a9,"5,422,214.972696736127979988",3.4850%,0.034850,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x1e6da5ed1586271ef004cefac694eedfe77bc7a9,False,0,False
7,7,0x06dad865936fa66abec9327667ae234f2dc6690c,"3,275,215.861619518032113566",2.1050%,0.021050,SYNC,SYNC,0xb6ff96b8a8d214544ca0dbc9b33f7ad6503efd32,ethereum,False,0x06dad865936fa66abec9327667ae234f2dc6690c,False,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23195,6,0xe63ff95e3112a46e84991f29b202548151460279,"3,443,433,168.561442546649439552",3.4434%,0.034434,ADAxMATIC,ADAxMATIC,0xbb2269B474F1fD0B45465B825DcE46a7a30567b0,binance,False,0xe63ff95e3112a46e84991f29b202548151460279,False,0,NaN
23196,7,0x31c4b2ca1e6e46e20602f5c884a6342a75aaf753,"2,202,723,600.000000010157429888",2.2027%,0.022027,ADAxMATIC,ADAxMATIC,0xbb2269B474F1fD0B45465B825DcE46a7a30567b0,binance,False,0x31c4b2ca1e6e46e20602f5c884a6342a75aaf753,False,0,NaN
23197,2,0xb620cd0216639044eed6f0caac2f3ffef9e88fff,"4,738,773,936.253045329",4.7388%,0.047388,BUSD TRAIN,TICKETS,0xEC33a3B177cf4Ce8E8D2810A0A0EDEC6B5ad92F0,binance,False,0xb620cd0216639044eed6f0caac2f3ffef9e88fff,False,0,NaN
23198,3,0xf02a3136290e1f6a118d41397ab2c9b16eab4851,"3,832,708,480.458668764",3.8327%,0.038327,BUSD TRAIN,TICKETS,0xEC33a3B177cf4Ce8E8D2810A0A0EDEC6B5ad92F0,binance,False,0xf02a3136290e1f6a118d41397ab2c9b16eab4851,False,0,NaN


## Non-Burn

In [6]:
lst = pd.read_csv(f'{RES}insider_trading/whale_list/list.csv')
lst = lst.loc[lst['address'] != '0x0000000000000000000000000000000000000dea']
lst = lst.loc[lst['address'] != '0x0000000000000000000000000000000000000000']
lst = lst.loc[lst['address'] != '0x0000000000000000000000000000000000000001']
lst.to_csv(f'{RES}insider_trading/whale_list/list.csv', index=False)

# Swap Spider

In [8]:
# panel = pd.read_csv(f'{RES}insider_trading/whale_list/list.csv')

# for chain in ['ethereum', 'binance', 'polygon', 'fantom', 'avalanche']:
#     for dex in chain_info(chain):
#         lst = panel.loc[panel['chain'] == chain].copy()
#         lst['status'] = -1
#         lst.to_csv(f'{WD}whale_swap_new/list/{dex}.csv', index=False)

In [9]:
class web_thread(threading.Thread):
    
    def __init__ (self, dex):
        threading.Thread.__init__(self)
        self.dex = dex
        
    def run(self):
        RD = '/home/team_defi/02_rdata/'
        WD = '/home/team_defi/03_wdata/'
        RES = '/home/team_defi/04_result/'
        
        lst = pd.read_csv(f'{WD}whale_swap_new/list/{self.dex}.csv')
        
        done = lst.loc[lst['status'] != -1].shape[0]
        total = lst.shape[0]
        
        for idx, row in lst.loc[lst['status'] == -1].iterrows():
            Address = row['Address']
            chain = row['chain']
            
            try:
                done += 1
                
                dfs = insider_transaction(address=Address, dex=self.dex)
                
                if len(dfs) == 0:
                    lst.loc[lst['Address'] == Address, 'status'] = 0
                    print(f'{Address}, {self.dex} no swap, {done}/{total}')
                else:
                    dfs.to_csv(f'{WD}whale_swap_new/csv/{Address}_{self.dex}.csv', index=False)
                    length = dfs.shape[0]
                    lst.loc[lst['Address'] == Address, 'status'] = length
                    print(f'{Address}, {self.dex} find {length} swap, {done}/{total}')
                
                lst.to_csv(f'{WD}whale_swap_new/list/{self.dex}.csv', index=False)
                
            except Exception as e:
                time.sleep(2)
                print(f'{Address}, {self.dex}, Error: {e}')
                
        print(f'{self.dex} done')

In [10]:
thread1 = web_thread('uniswapv3_ethereum')
thread2 = web_thread('uniswapv2_ethereum')
thread3 = web_thread('sushiswap_ethereum')

thread4 = web_thread('pancake_binance')
thread5 = web_thread('biswap_binance')
thread6 = web_thread('apeswap_binance')
thread7 = web_thread('babyswap_binance')
thread8 = web_thread('jetswap_binance')

thread9 = web_thread('traderjoe_avalanche')
thread10 = web_thread('pangolin_avalanche')

thread11 = web_thread('sushiswap_polygon')
thread12 = web_thread('apeswap_polygon')
thread13 = web_thread('jetswap_polygon')

In [11]:
Threads = [
           thread1, 
           thread2, 
           thread3,
           thread4,
           thread5,
           thread6,
           thread7,
           thread8,
           thread9,
           thread10,
           thread11,
           thread12,
           thread13,
          ]
for t in Threads:
    t.start()
for t in Threads:
    t.join()

traderjoe_avalanche done
pangolin_avalanche done
sushiswap_polygon done
apeswap_polygon done
jetswap_polygon done
babyswap_binance done
biswap_binance done
apeswap_binance done
jetswap_binance done
sushiswap_ethereum done
0xf07704777d6bc182bf2c67fbda48913169b84983, uniswapv3_ethereum, Error: 'data'
0x98fbdbb901b9067aec563b59e06a979a24861004, uniswapv2_ethereum find 1492 swap, 14118/14231
0x4e8a511f117c80b97db153fd0adf77986078e175, uniswapv2_ethereum, Error: 'data'
0x1e83fac456a79b575a6f3045b88a4b9a98b1b8d2, pancake_binance, Error: 'data'
0x60b86af869f23aeb552fb7f3cabd11b829f6ab2f, uniswapv3_ethereum, Error: 'data'
uniswapv3_ethereum done
0xf07704777d6bc182bf2c67fbda48913169b84983, uniswapv2_ethereum, Error: 'data'
0xf67eca83da34ee6fb422640cfd7083482d00af96, uniswapv2_ethereum find 1305 swap, 14121/14231
0x7be8202dbebe7e7751e415097417f5bc7b315fd3, uniswapv2_ethereum find 1073 swap, 14122/14231
0xae857493e6296beec24a386ee8ccc914010a16d2, pancake_binance, Error: 'data'
0x7a1b82fe48a68b9bc

# Home Token

In [13]:
for chain in ['ethereum', 'binance', 'avalanche', 'polygon', 'fantom']:
    dfs = []
    
    for dex in chain_info(chain):

        
        lst = pd.read_csv(f'{WD}whale_swap_new/list/{dex}.csv')
        total = lst.loc[lst['status'] > 0].shape[0]
        num = 0
        
        for idx, row in lst.loc[lst['status'] > 0].iterrows():
            num += 1
            print(f'{dex}, {num}/{total}')
            Address = row['Address']
            contractAddress = row['contractAddress']
            
            df = pd.read_csv(f'{WD}whale_swap_new/csv/{Address}_{dex}.csv')
            df = home_token_filter(df, contractAddress)
            df = df.loc[df['home_token'] == True]
            
            if df.shape[0] != 0:
                df['dex'] = dex
                df['chain'] = chain
                df['insider_Address'] = Address
                df['insider_contractAddress'] = contractAddress
                dfs.append(df)
                
                
    if len(dfs) != 0:
        dfs = pd.concat(dfs)
        dfs.to_csv(f'{WD}whale_swap_new/panel/{chain}.csv', index=False)

uniswapv3_ethereum, 1/9236
uniswapv3_ethereum, 2/9236
uniswapv3_ethereum, 3/9236
uniswapv3_ethereum, 4/9236
uniswapv3_ethereum, 5/9236
uniswapv3_ethereum, 6/9236
uniswapv3_ethereum, 7/9236
uniswapv3_ethereum, 8/9236
uniswapv3_ethereum, 9/9236
uniswapv3_ethereum, 10/9236
uniswapv3_ethereum, 11/9236
uniswapv3_ethereum, 12/9236
uniswapv3_ethereum, 13/9236
uniswapv3_ethereum, 14/9236
uniswapv3_ethereum, 15/9236
uniswapv3_ethereum, 16/9236
uniswapv3_ethereum, 17/9236
uniswapv3_ethereum, 18/9236
uniswapv3_ethereum, 19/9236
uniswapv3_ethereum, 20/9236
uniswapv3_ethereum, 21/9236
uniswapv3_ethereum, 22/9236
uniswapv3_ethereum, 23/9236
uniswapv3_ethereum, 24/9236
uniswapv3_ethereum, 25/9236
uniswapv3_ethereum, 26/9236
uniswapv3_ethereum, 27/9236
uniswapv3_ethereum, 28/9236
uniswapv3_ethereum, 29/9236
uniswapv3_ethereum, 30/9236
uniswapv3_ethereum, 31/9236
uniswapv3_ethereum, 32/9236
uniswapv3_ethereum, 33/9236
uniswapv3_ethereum, 34/9236
uniswapv3_ethereum, 35/9236
uniswapv3_ethereum, 36/9236
u

# Check

In [14]:
dfs = []
for chain in ['ethereum', 'binance']:   
    df = pd.read_csv(f'{WD}whale_swap_new/panel/{chain}.csv')
    df
    dfs.append(df)
    
dfs = pd.concat(dfs)
dfs.to_csv(f'{WD}whale_swap_new/panel/panel.csv', index=False)

,amountUSD,id,logIndex,origin,recipient,sender,sqrtPriceX96,tick,timestamp,token0_name,token1_name,token0_id,token1_id,transaction_id,amount0Out,amount0In,amount1Out,amount1In,home_token,dex,chain,insider_Address,insider_contractAddress,from,to
0,992.960723,0x0b49eaa63d67bcb8c9cd1375854feea9b47621db48387d2aa90987ccb048d95c#5022,349,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,430934722968841228409154365,-104288.0,2021-07-14 14:27:23,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x0b49eaa63d67bcb8c9cd1375854feea9b47621db48387d2aa90987ccb048d95c,1.670782e+04,0.000000,0.000000,0.495625,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN
1,113.388570,0x15526257752dbbdc0959345822d4c64f82d0b5fa11293df6fa690071e52ab5b9#5193,184,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,454242565318554607628393138,-103235.0,2021-07-15 18:50:05,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x15526257752dbbdc0959345822d4c64f82d0b5fa11293df6fa690071e52ab5b9,1.803967e+03,0.000000,0.000000,0.059475,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN
2,451.317373,0x1ac711c16d8bd092e258a266b3cf2e15a78049c8e39600f49d349f295ca96ae2#5070,270,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,421439188110885901054537524,-104734.0,2021-07-14 19:13:13,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x1ac711c16d8bd092e258a266b3cf2e15a78049c8e39600f49d349f295ca96ae2,8.034505e+03,0.000000,0.000000,0.227988,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN
3,31235.114543,0x1c7ad5ad9a263b9b5b7ad09eecaadb2d74855ae3cdc9391b5be85c69978769e6#2728,160,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xdef1c0ded9bec7f1a1670819833240f027b25eff,540565432494541425309469623,-99755.0,2021-06-30 19:44:14,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x1c7ad5ad9a263b9b5b7ad09eecaadb2d74855ae3cdc9391b5be85c69978769e6,0.000000e+00,300000.000000,14.021706,0.000000,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN
4,94.835360,0x20542f4aae1362de27198940f2d829d806fe40bb775ea78dcfbe779cf5a61cbf#5204,247,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,449521301971896931887340712,-103444.0,2021-07-15 19:40:40,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x20542f4aae1362de27198940f2d829d806fe40bb775ea78dcfbe779cf5a61cbf,1.535041e+03,0.000000,0.000000,0.049563,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31622,11.030563,0xf22f60d6f635deed41123f4ca2b658ddf5f39ceca3227e2c954259bc348cded1-2,274,NaN,NaN,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,NaN,NaN,2022-04-19 06:52:16,Narfex,SushiToken,0x155cd154b4c3afc2719601b617e52526a520d301,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,0xf22f60d6f635deed41123f4ca2b658ddf5f39ceca3227e2c954259bc348cded1,4.000000e+05,4107.117647,0.000000,3.364816,True,sushiswap_ethereum,ethereum,0x9e8db3942797d2578f48caf5663eb22e286ad84b,0x155cd154b4c3afc2719601b617e52526a520d301,NaN,0x9e8db3942797d2578f48caf5663eb22e286ad8

,amount0In,amount0Out,amount1In,amount1Out,amountUSD,from,id,logIndex,sender,timestamp,to,token0_name,token1_name,token0_id,token1_id,transaction_id,home_token,dex,chain,insider_Address,insider_contractAddress
0,2.142616,0.000000,0.000000,3.000000e+06,710.716620,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,0x30ffe8cfedd98a47391c49fd3b2d9c9a2bda52b96bf15ca4ce8aab1037aaa79e-1,529,0x10ed43c718714eb63d5aa57b78b54704e256024e,2021-07-31 19:40:31,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,Wrapped BNB,WAIFU INU,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe338946b3fd443e5019079c439a9830ce762c475,0x30ffe8cfedd98a47391c49fd3b2d9c9a2bda52b96bf15ca4ce8aab1037aaa79e,True,pancake_binance,binance,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,0xe338946b3fd443e5019079c439a9830ce762c475
1,1.308880,0.000000,0.000000,1.000000e+07,433.403923,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,0x997df0f01aebd0b3f80a8a8e20f6644768c1983a9f136e83d0ad8dd35b012bca-1,939,0x10ed43c718714eb63d5aa57b78b54704e256024e,2021-08-05 16:42:19,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,Wrapped BNB,WAIFU INU,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe338946b3fd443e5019079c439a9830ce762c475,0x997df0f01aebd0b3f80a8a8e20f6644768c1983a9f136e83d0ad8dd35b012bca,True,pancake_binance,binance,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,0xe338946b3fd443e5019079c439a9830ce762c475
2,3.567947,0.000000,0.000000,4.466176e+06,1180.781435,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,0xbe798650b7f30249a5517f8d0c04132516f9da13b6799ba8dd5901e3cb7a4b79-1,442,0x10ed43c718714eb63d5aa57b78b54704e256024e,2021-07-31 20:14:26,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,Wrapped BNB,WAIFU INU,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe338946b3fd443e5019079c439a9830ce762c475,0xbe798650b7f30249a5517f8d0c04132516f9da13b6799ba8dd5901e3cb7a4b79,True,pancake_binance,binance,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,0xe338946b3fd443e5019079c439a9830ce762c475
3,1.395713,0.000000,0.000000,2.000000e+06,462.886045,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,0xc993ec3718039c8cbdf303550e44b7a29c3d8e024720de96d4851a7937fdc9e6-1,120,0x10ed43c718714eb63d5aa57b78b54704e256024e,2021-07-31 19:35:34,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,Wrapped BNB,WAIFU INU,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe338946b3fd443e5019079c439a9830ce762c475,0xc993ec3718039c8cbdf303550e44b7a29c3d8e024720de96d4851a7937fdc9e6,True,pancake_binance,binance,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,0xe338946b3fd443e5019079c439a9830ce762c475
4,3.707983,0.000000,0.000000,5.000000e+06,1230.227091,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,0xe770fdd11f871bd6be8d3b4ef1356423a378b7a307e336bc55826b7e4bb71053-1,644,0x10ed43c718714eb63d5aa57b78b54704e256024e,2021-07-31 19:43:16,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,Wrapped BNB,WAIFU INU,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe338946b3fd443e5019079c439a9830ce762c475,0xe770fdd11f871bd6be8d3b4ef1356423a378b7a307e336bc55826b7e4bb71053,True,pancake_binance,binance,0x06b3d19744e06ee5a0e08fbdf99c1c35cae1cdbb,0xe338946b3fd443e5019079c439a9830ce762c475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15394,0.000000,0.000001,11394.147974,0.000000e+00,0.000343,0x3c9c5b4ab09ebee2f447d26704f3970871c7a9fb,0xe349a985d189cf49037a4a7c46826d877f4c7493ff802a3d96b37b9ea5058866-2,185,0x325e343f1de602396e256b67efd1f61c3a6b38bd,2022-07-06 10:26:44,0x325e343f1de602396e256b67efd1f61c3a6b38bd,Wrapped BNB,Baby o Baby,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe798e8f65e4252a55ffc552c17c576ba9b05c15b,0xe349a985d189cf49037a4a7c46826d877f4c7493ff802a3d96b37b9ea5058866,True,babyswap_binance,binance,0x3c9c5b4ab09ebee2f447d26704f3970871c7a9fb,0xE798e8f65E4252A55FFc552C17C576Ba9B05C15b
15395,0.000000,0.000005,38014.656121,0.000000e+00,0.001145,0x3c9c5b4ab09ebee2f447d26704f3970871c7a9fb,0xe349a985d189cf49037a4a7c46826d877f4c7493ff802a3d96b37b9ea5058866-3,202,0x325e343f1de602396e256b67efd1f61c3a6b38bd,2022-07-06 10:26:44,0x36ae10a4d16311959b607ee98bc4a8a653a33b1f,Wrapped BNB,Baby o B

# Liquidity Provision

In [ ]:
df = pd.read_csv(f'{WD}whale_swap_new/panel/panel.csv')
total = df.shape[0]
num = 0
df['lp'] = np.nan

for idx, row in df.iterrows():
    num += 1
    
    transaction_id = row['transaction_id']
    chain = row['chain']
    try:
        status = liquidity_provision(transaction_id, chain)
        if status == True:
            df.loc[df['transaction_id'] == transaction_id, 'lp'] = True
            print(f'{transaction_id} is lp, {num}/{total}.')

        else:
            df.loc[df['transaction_id'] == transaction_id, 'lp'] = False
            print(f'{transaction_id} is not lp, {num}/{total}.')
    except Exception as e:
        print(f'error:{e}')

/tmp/ipykernel_450521/3830462909.py:1: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{WD}whale_swap_new/panel/panel.csv')


0x0b49eaa63d67bcb8c9cd1375854feea9b47621db48387d2aa90987ccb048d95c is not lp, 1/47026.
0x15526257752dbbdc0959345822d4c64f82d0b5fa11293df6fa690071e52ab5b9 is not lp, 2/47026.
0x1ac711c16d8bd092e258a266b3cf2e15a78049c8e39600f49d349f295ca96ae2 is not lp, 3/47026.
0x1c7ad5ad9a263b9b5b7ad09eecaadb2d74855ae3cdc9391b5be85c69978769e6 is not lp, 4/47026.
0x20542f4aae1362de27198940f2d829d806fe40bb775ea78dcfbe779cf5a61cbf is not lp, 5/47026.
0x29fef739e6cdc797d884816c0df2594d08e80c66cd8b15698cbac5f4800342c8 is not lp, 6/47026.
0x2b522ab1794011af1b9b14eedc8842b5b1b17e2f09a36c9ca43bca8ced81d4ad is not lp, 7/47026.
0x2e9754c7fb40a83da9eb03910cd5f891b2d5764c796b6dfb1622d042d3ba63f9 is not lp, 8/47026.
0x30d3f459cb86e136ae9aee7699e84e1b1b2c6faaa3a0304f00bda59c0023f3f4 is not lp, 9/47026.
0x3ec9e1de25f3d368240a823bff5a2d4f7bc2121c3b2204a9db8886ae33bf01de is not lp, 10/47026.
0x41109ae4e081fc23c5c790dacd7e7781f75ccfb0e9879a87770d4e3a702803ec is not lp, 11/47026.
0x42d66bb7b6976383e9c855429e20689f27df832

In [24]:
df = df.loc[df['lp']==False]
df
df.to_csv(f'{WD}whale_swap_new/panel/panel_exclude_lp.csv', index=False)

,amountUSD,id,logIndex,origin,recipient,sender,sqrtPriceX96,tick,timestamp,token0_name,token1_name,token0_id,token1_id,transaction_id,amount0Out,amount0In,amount1Out,amount1In,home_token,dex,chain,insider_Address,insider_contractAddress,from,to,lp
0,992.960723,0x0b49eaa63d67bcb8c9cd1375854feea9b47621db48387d2aa90987ccb048d95c#5022,349,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,430934722968841228409154365,-104288.0,2021-07-14 14:27:23,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x0b49eaa63d67bcb8c9cd1375854feea9b47621db48387d2aa90987ccb048d95c,16707.818423,0.000000,0.000000e+00,0.495625,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN,False
1,113.388570,0x15526257752dbbdc0959345822d4c64f82d0b5fa11293df6fa690071e52ab5b9#5193,184,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,454242565318554607628393138,-103235.0,2021-07-15 18:50:05,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x15526257752dbbdc0959345822d4c64f82d0b5fa11293df6fa690071e52ab5b9,1803.966671,0.000000,0.000000e+00,0.059475,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN,False
2,451.317373,0x1ac711c16d8bd092e258a266b3cf2e15a78049c8e39600f49d349f295ca96ae2#5070,270,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,421439188110885901054537524,-104734.0,2021-07-14 19:13:13,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x1ac711c16d8bd092e258a266b3cf2e15a78049c8e39600f49d349f295ca96ae2,8034.504798,0.000000,0.000000e+00,0.227988,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN,False
3,31235.114543,0x1c7ad5ad9a263b9b5b7ad09eecaadb2d74855ae3cdc9391b5be85c69978769e6#2728,160,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xdef1c0ded9bec7f1a1670819833240f027b25eff,540565432494541425309469623,-99755.0,2021-06-30 19:44:14,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x1c7ad5ad9a263b9b5b7ad09eecaadb2d74855ae3cdc9391b5be85c69978769e6,0.000000,300000.000000,1.402171e+01,0.000000,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN,False
4,94.835360,0x20542f4aae1362de27198940f2d829d806fe40bb775ea78dcfbe779cf5a61cbf#5204,247,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,449521301971896931887340712,-103444.0,2021-07-15 19:40:40,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x20542f4aae1362de27198940f2d829d806fe40bb775ea78dcfbe779cf5a61cbf,1535.040843,0.000000,0.000000e+00,0.049563,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47021,0.000343,0xe349a985d189cf49037a4a7c46826d877f4c7493ff802a3d96b37b9ea5058866-2,185,NaN,NaN,0x325e343f1de602396e256b67efd1f61c3a6b38bd,NaN,NaN,2022-07-06 10:26:44,Wrapped BNB,Baby o Baby,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe798e8f65e4252a55ffc552c17c576ba9b05c15b,0xe349a985d189cf49037a4a7c46826d877f4c7493ff802a3d96b37b9ea5058866,0.000001,0.000000,0.000000e+00,11394.147974,True,babyswap_binance,binance,0x3c9c5b4ab09ebee2f447d26704f3970871c7a9fb,0xE798e8f65E4252A55FFc552C17C576Ba9

# Variables

In [26]:
df = pd.read_csv(f'{WD}whale_swap_new/panel/panel_exclude_lp.csv')
df.drop_duplicates('id', inplace=True)

df = set_buy(df=df)
df.to_csv(f'{WD}whale_swap_new/panel/panel_buy.csv', index=False)

/tmp/ipykernel_450521/370901579.py:1: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{WD}whale_swap_new/panel/panel_exclude_lp.csv')


In [27]:
df

,amountUSD,id,logIndex,origin,recipient,sender,sqrtPriceX96,tick,timestamp,token0_name,token1_name,token0_id,token1_id,transaction_id,amount0Out,amount0In,amount1Out,amount1In,home_token,dex,chain,insider_Address,insider_contractAddress,from,to,lp,buy
0,992.960723,0x0b49eaa63d67bcb8c9cd1375854feea9b47621db48387d2aa90987ccb048d95c#5022,349,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,430934722968841228409154365,-104288.0,2021-07-14 14:27:23,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x0b49eaa63d67bcb8c9cd1375854feea9b47621db48387d2aa90987ccb048d95c,16707.818423,0.000000,0.000000e+00,0.495625,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN,False,992.960723
1,113.388570,0x15526257752dbbdc0959345822d4c64f82d0b5fa11293df6fa690071e52ab5b9#5193,184,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,454242565318554607628393138,-103235.0,2021-07-15 18:50:05,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x15526257752dbbdc0959345822d4c64f82d0b5fa11293df6fa690071e52ab5b9,1803.966671,0.000000,0.000000e+00,0.059475,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN,False,113.388570
2,451.317373,0x1ac711c16d8bd092e258a266b3cf2e15a78049c8e39600f49d349f295ca96ae2#5070,270,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,421439188110885901054537524,-104734.0,2021-07-14 19:13:13,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x1ac711c16d8bd092e258a266b3cf2e15a78049c8e39600f49d349f295ca96ae2,8034.504798,0.000000,0.000000e+00,0.227988,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN,False,451.317373
3,31235.114543,0x1c7ad5ad9a263b9b5b7ad09eecaadb2d74855ae3cdc9391b5be85c69978769e6#2728,160,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xdef1c0ded9bec7f1a1670819833240f027b25eff,540565432494541425309469623,-99755.0,2021-06-30 19:44:14,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x1c7ad5ad9a263b9b5b7ad09eecaadb2d74855ae3cdc9391b5be85c69978769e6,0.000000,300000.000000,1.402171e+01,0.000000,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN,False,-31235.114543
4,94.835360,0x20542f4aae1362de27198940f2d829d806fe40bb775ea78dcfbe779cf5a61cbf#5204,247,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xdef1c0ded9bec7f1a1670819833240f027b25eff,449521301971896931887340712,-103444.0,2021-07-15 19:40:40,Zort Coin,Wrapped Ether,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x20542f4aae1362de27198940f2d829d806fe40bb775ea78dcfbe779cf5a61cbf,1535.040843,0.000000,0.000000e+00,0.049563,True,uniswapv3_ethereum,ethereum,0x489c6246941fecacae2a33d8ee7ba207a7526e95,0x825cd4201f8a2bbb1a69668eac4e5fa71283273d,NaN,NaN,False,94.835360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46597,0.000343,0xe349a985d189cf49037a4a7c46826d877f4c7493ff802a3d96b37b9ea5058866-2,185,NaN,NaN,0x325e343f1de602396e256b67efd1f61c3a6b38bd,NaN,NaN,2022-07-06 10:26:44,Wrapped BNB,Baby o Baby,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xe798e8f65e4252a55ffc552c17c576ba9b05c15b,0xe349a985d189cf49037a4a7c46826d877f4c7493ff802a3d96b37b9ea5058866,0.000001,0.000000,0.000000e+00,11394.147974,True,babyswap_binance,binance,0x3c9c5b4ab09

In [ ]:
for after in [True, False]:
    for day in [1, 2, 3]:
        if after == True: df[f'ret_{str(day)}'] = np.nan
        if after == False: df[f'ret_-{str(day)}'] = np.nan
total = df.shape[0]
num = 0

for idx, row in df.iterrows():
    try:
        num += 1
        insider_contractAddress = row['insider_contractAddress']
        timestamp = row['timestamp']
        index = row['id']
        chain = row['chain']
        print(f'{num}/{total}')
        for after in [True, False]:
            for day in [1, 2, 3]:
                
                if after == True: df.loc[df['id'] == index, f'ret_{str(day)}'] = token_return(after=after, 
                                                                                              day=day, 
                                                                                              contract=insider_contractAddress, 
                                                                                              chain=chain,
                                                                                              timestamp=timestamp)
                if after == False: df.loc[df['id'] == index, f'ret_-{str(day)}'] = token_return(after=after, 
                                                                                              day=day, 
                                                                                              contract=insider_contractAddress, 
                                                                                              chain=chain,
                                                                                              timestamp=timestamp)
    except Exception as e:
        print(f'{num}/{total}:Error{e}')

1/45188
2/45188
3/45188
4/45188
5/45188
6/45188
7/45188
8/45188
9/45188
10/45188
11/45188
12/45188
13/45188
14/45188
15/45188
16/45188
17/45188
18/45188
19/45188
20/45188
21/45188
22/45188
23/45188
24/45188
25/45188
26/45188
27/45188
28/45188
29/45188
30/45188
31/45188
32/45188
33/45188
34/45188
35/45188
36/45188
37/45188
38/45188
39/45188
40/45188
41/45188
42/45188
43/45188
44/45188
45/45188
46/45188
47/45188
48/45188
49/45188
50/45188
51/45188
52/45188
53/45188
54/45188
55/45188
56/45188
57/45188
58/45188
59/45188
60/45188
61/45188
62/45188
63/45188
64/45188
65/45188
66/45188
67/45188
68/45188
69/45188
70/45188
71/45188
72/45188
73/45188
74/45188
75/45188
76/45188
77/45188
78/45188
79/45188
80/45188
81/45188
82/45188
83/45188
84/45188
85/45188
86/45188
87/45188
88/45188
89/45188
90/45188
91/45188
92/45188
93/45188
94/45188
95/45188
96/45188
97/45188
98/45188
99/45188
100/45188
101/45188
102/45188
103/45188
104/45188
105/45188
106/45188
107/45188
108/45188
109/45188
110/45188
111/4518